# EDA and ETL Process
This file presents the data exploration and data cleaning process. 

## 1. Data Description 
The dataset is from two data sources: one records the [load interconnection capability](https://transmission.bpa.gov/Business/Operations/Charts/triCities.txt) in tri-cities, another records a project "[Ashe 230kV Main Bus Voltage](https://transmission.bpa.gov/Business/Operations/Charts/ashe.txt)" in tri-cities. They are both updated every 5 minutes, and stored in a txt file.

In [1]:
import requests

# Ashe 230kV Main Bus Voltage Dataset
url1 = "https://transmission.bpa.gov/Business/Operations/Charts/ashe.txt"
req1 = requests.get(url1)
req1.raise_for_status()
text1 = req1.text
print(text1[:500])
print('...')
print(text1[-500:])

Ashe 230kV Main Bus Voltage
at 5-minute intervals, last 7 days
Dates: 04Dec2021 - 11Dec2021 (last updated 10Dec2021 09:10:37) Pacific Time
Based on 5-min readings from the BPA SCADA system for point 2022
BPA/Technical Operations (TOT-OpInfo@bpa.gov)

Date/Time       	
12/04/2021 00:00	240.396
12/04/2021 00:05	240.396
12/04/2021 00:10	240.552
12/04/2021 00:15	240.084
12/04/2021 00:20	240.24
12/04/2021 00:25	240.552
12/04/2021 00:30	240.552
12/04/2021 00:35	240.396
12/04/2021 00:40	
...
:45	
12/10/2021 21:50	
12/10/2021 21:55	
12/10/2021 22:00	
12/10/2021 22:05	
12/10/2021 22:10	
12/10/2021 22:15	
12/10/2021 22:20	
12/10/2021 22:25	
12/10/2021 22:30	
12/10/2021 22:35	
12/10/2021 22:40	
12/10/2021 22:45	
12/10/2021 22:50	
12/10/2021 22:55	
12/10/2021 23:00	
12/10/2021 23:05	
12/10/2021 23:10	
12/10/2021 23:15	
12/10/2021 23:20	
12/10/2021 23:25	
12/10/2021 23:30	
12/10/2021 23:35	
12/10/2021 23:40	
12/10/2021 23:45	
12/10/2021 23:50	
12/10/2021 23:55	



In [2]:
# Tri-Cities Load/Import/Gen
url2 = "https://transmission.bpa.gov/Business/Operations/Charts/triCities.txt"
req2 = requests.get(url2, timeout=0.5)
req2.raise_for_status()
text2 = req2.text
print(text2[:500])
print('...')
print(text2[-500:])

Tri-Cities Load/Import/Gen
at 5-minute intervals, last 7 days
Dates: 04Dec2021 - 11Dec2021 (last updated 10Dec2021 09:10:41) Pacific Time
Based on 5-min MW readings from the BPA SCADA system for points 98222, 98223, 98220
BPA/Technical Operations (TOT-OpInfo@bpa.gov)

Date/Time       	Import	Load	Generation
12/04/2021 00:00	365.9039	456.0815	90.17763
12/04/2021 00:05	366.6361	455.9711	89.33501
12/04/2021 00:10	364.2567	453.2793	89.02254
12/04/2021 00:15	364.5209	452.8102	88.28934
12/0
...
0/2021 22:00			
12/10/2021 22:05			
12/10/2021 22:10			
12/10/2021 22:15			
12/10/2021 22:20			
12/10/2021 22:25			
12/10/2021 22:30			
12/10/2021 22:35			
12/10/2021 22:40			
12/10/2021 22:45			
12/10/2021 22:50			
12/10/2021 22:55			
12/10/2021 23:00			
12/10/2021 23:05			
12/10/2021 23:10			
12/10/2021 23:15			
12/10/2021 23:20			
12/10/2021 23:25			
12/10/2021 23:30			
12/10/2021 23:35			
12/10/2021 23:40			
12/10/2021 23:45			
12/10/2021 23:50			
12/10/2021 23:55			



## 2. Convert the Raw Data to Dataframe
We use pandas to make the transformation, and give the voltage column from the Ashe 230kV Main Bus Voltage dataset a column name.

In [4]:
import pandas as pd
from io import StringIO

df1 = pd.read_csv(StringIO(text1), skiprows=6, delimiter='\t')
df1.columns = df1.columns.str.strip()             # remove space in columns name
df1['Datetime'] = pd.to_datetime(df1['Date/Time'])
df1.drop(columns=['Date/Time'], axis=1, inplace=True)
df1.dropna(inplace=True)  
df1.columns = ['vol_value', 'Datetime']
df1.head()

,vol_value,Datetime
0,240.396,2021-12-04 00:00:00
1,240.396,2021-12-04 00:05:00
2,240.552,2021-12-04 00:10:00
3,240.084,2021-12-04 00:15:00
4,240.240,2021-12-04 00:20:00


In [5]:
# use the same method to get the second df
df2 = pd.read_csv(StringIO(text2), skiprows=6, delimiter='\t')
df2.columns = df2.columns.str.strip()             # remove space in columns name
df2['Datetime'] = pd.to_datetime(df2['Date/Time'])
df2.drop(columns=['Date/Time'], axis=1, inplace=True)
df2.dropna(inplace=True)  
df2.head()

,Import,Load,Generation,Datetime
0,365.9039,456.0815,90.17763,2021-12-04 00:00:00
1,366.6361,455.9711,89.33501,2021-12-04 00:05:00
2,364.2567,453.2793,89.02254,2021-12-04 00:10:00
3,364.5209,452.8102,88.28934,2021-12-04 00:15:00
4,362.2499,450.5781,88.32825,2021-12-04 00:20:00


In [6]:
# Two datasets are joined together
df2.drop(['Datetime'], axis=1, inplace=True)
df = pd.concat([df1, df2], axis=1)
# change the order for the columns
df = df[['Datetime', 'vol_value', 'Import', 'Load', 'Generation']] 
df.head()

,Datetime,vol_value,Import,Load,Generation
0,2021-12-04 00:00:00,240.396,365.9039,456.0815,90.17763
1,2021-12-04 00:05:00,240.396,366.6361,455.9711,89.33501
2,2021-12-04 00:10:00,240.552,364.2567,453.2793,89.02254
3,2021-12-04 00:15:00,240.084,364.5209,452.8102,88.28934
4,2021-12-04 00:20:00,240.240,362.2499,450.5781,88.32825


## 3. Save and Update the Dataset into Database: MongoDB
We locate the data document, update it if exists or insert the rows into the dataset.

In [8]:
import pymongo

client = pymongo.MongoClient()

In [10]:
db = client.get_database("voltage")
collection = db.get_collection("voltage")
update_count = 0
for record in df.to_dict('records'): # convert the dataframe into dict
    result = collection.replace_one(
        filter={'Datetime': record['Datetime']},    # locate the document if exists
        replacement=record,                         # latest document
        upsert=True)                                # update if exists, insert if not
    if result.matched_count > 0:
        update_count += 1
print("rows =", df.shape[0], "update =",update_count, "insert =",df.shape[0]-update_count)

rows = 1839 update = 1839 insert = 0
